# **2. Estimación y Selección**

In [36]:
import numpy as np
import pandas as pd
import math

def obtener_datos_fase1(ruta_archivo):
    df = pd.read_csv(ruta_archivo)
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date')
    serie = df['Monthly Mean Total Sunspot Number'].values
    return serie
try:
    serie_solar = obtener_datos_fase1('/content/train.csv')
except:
    pass

Los pasos a seguir son los siguientes:

1. Diferenciación Manual

2. Cálculo de Errores

3. Cálculo del Gradiente

4. Algoritmo de Estimación

5. Criterio de Selección (AIC)

6. Ejecución y Selección Final (Selección)

### 1. Diferenciación Manual

Ante la evidencia de un comportamiento no estacionario en la media de la serie original $\{Z_t\}$, asociado a la presencia de raíces unitarias, resulta imprescindible aplicar una transformación lineal mediante el método de diferenciación para inducir la estacionariedad.

Para formalizar este proceso, se define el **operador de retardo** $B$ tal que su aplicación sobre una observación en el tiempo $t$ resulta en el valor del periodo inmediatamente anterior:

$$B z_t = z_{t-1}$$

A partir de este, se establece el **operador diferencia** $\nabla$, definido como:

$$\nabla z_t = (1 - B)z_t = z_t - z_{t-1}$$

Para generalizar este procedimiento a un proceso integrado de orden $d$, la serie transformada estacionaria $\{y_t\}$ se obtiene mediante la aplicación sucesiva de este operador:

$$y_t = \nabla^d z_t = (1 - B)^d z_t$$

La aplicación de esta transformación elimina las tendencias estocásticas presentes en el proceso generador de datos, produciendo una nueva serie que satisface las condiciones de estabilidad requeridas para la estimación paramétrica.

In [37]:
def diferenciar_manual(datos, d):
    serie = np.array(datos, dtype=float)
    for _ in range(d):
        n = len(serie)
        nueva_serie = np.zeros(n - 1)
        for t in range(1, n):
            nueva_serie[t-1] = serie[t] - serie[t-1]
        serie = nueva_serie
    return serie

### 2. Cálculo de Errores y Definición del Modelo ARMA

Una vez garantizada la condición de estacionariedad en la serie, el proceso estocástico resultante $\{z_t\}$ se representa mediante un modelo Autorregresivo de Medias Móviles, denotado como $ARMA(p,q)$. La estructura lineal que rige el comportamiento temporal de la serie se define formalmente como:

$$z_t = c + \sum_{i=1}^{p}\phi_{i}z_{t-i} + a_t - \sum_{j=1}^{q}\theta_{j}a_{t-j}$$

Donde:
* $\phi_i$ son los coeficientes autorregresivos.
* $\theta_j$ son los coeficientes de medias móviles.
* $c$ es la constante del modelo.
* $\{a_t\}$ representa el proceso de innovaciones (ruido blanco) con media cero.

Para proceder con la estimación de los parámetros, es necesario aislar el término de error para cuantificar la discrepancia entre la observación real y el valor esperado por el modelo. Reordenando los términos de la ecuación general, la innovación en el tiempo $t$ se expresa como:

$$a_t = z_t - \left( c + \sum_{i=1}^{p}\phi_{i}z_{t-i} - \sum_{j=1}^{q}\theta_{j}a_{t-j} \right)$$

El cálculo de esta secuencia de residuos se efectúa de manera recursiva. Dado que las innovaciones pasadas son desconocidas al inicio del periodo de observación, se adopta el supuesto de **estimación condicional**, estableciendo la condición inicial de que $a_t = 0$ para todo $t \le 0$. La suma de los cuadrados de estos errores calculados ($SSE$) constituye la función objetivo que se buscará minimizar posteriormente.

In [38]:
def calcular_sse_manual(params, serie, p, q):
    T = len(serie)
    c = params[0]
    phis = params[1 : p+1]
    thetas = params[p+1 : p+1+q]

    a = np.zeros(T)
    sse = 0.0
    k = max(p, q)

    for t in range(k, T):
        comp_ar = 0.0
        for i in range(p):
            comp_ar += phis[i] * serie[t - (i + 1)]

        comp_ma = 0.0
        for j in range(q):
            comp_ma += thetas[j] * a[t - (j + 1)]

        prediccion = c + comp_ar - comp_ma
        error = serie[t] - prediccion
        a[t] = error
        sse += error**2

    return sse

### 3. Cálculo del Gradiente

El problema de estimación de los parámetros del modelo $\beta = (c, \phi, \theta)$ se formula como un problema de optimización no lineal, cuyo objetivo es encontrar el vector de coeficientes que minimice la función de pérdida definida por la suma de cuadrados de los residuos, $S(\beta)$.

Para resolver este problema mediante métodos iterativos de optimización, es indispensable determinar la dirección de máximo descenso de la función objetivo en el espacio de parámetros. Esta dirección está dada por el vector gradiente $\nabla S(\beta)$, el cual se compone de las derivadas parciales de la función de pérdida respecto a cada uno de los coeficientes del modelo:

$$\nabla S(\beta) = \left[ \frac{\partial S}{\partial c}, \frac{\partial S}{\partial \phi_1}, \dots, \frac{\partial S}{\partial \phi_p}, \frac{\partial S}{\partial \theta_1}, \dots, \frac{\partial S}{\partial \theta_q} \right]^T$$

Dada la complejidad algebraica derivada de la recursividad en los términos de media móvil, la evaluación analítica de estas derivadas resulta impracticable. Por consiguiente, se emplea el método de aproximación numérica por diferencias finitas. La derivada parcial respecto al $i$-ésimo parámetro se aproxima mediante el cociente incremental:

$$\frac{\partial S}{\partial \beta_i} \approx \frac{S(\beta + h \cdot e_i) - S(\beta)}{h}$$

Donde $h$ representa una perturbación infinitesimal y $e_i$ es el vector unitario en la dirección del parámetro $i$. Este cálculo permite cuantificar la sensibilidad del error ante variaciones marginales en los coeficientes, proporcionando la información necesaria para actualizar las estimaciones.

In [39]:
def calcular_gradiente_manual(params, serie, p, q):
    epsilon = 1e-5
    grads = np.zeros(len(params))
    base_sse = calcular_sse_manual(params, serie, p, q)

    for i in range(len(params)):
        p_copy = params.copy()
        p_copy[i] += epsilon
        new_sse = calcular_sse_manual(p_copy, serie, p, q)
        grads[i] = (new_sse - base_sse) / epsilon

    return grads

### 4. Algoritmo de Estimación

La estimación de los coeficientes del modelo se realiza bajo el principio de Máxima Verosimilitud. Asumiendo que los errores siguen una distribución normal, la maximización de la función de verosimilitud es equivalente a la minimización de la Suma de Cuadrados de los Residuos ($SSR$). Se define el problema de optimización como:

$$\min_{\beta} S(\beta) = \sum_{t=1}^{T} a_t^2(\beta)$$

Donde $\beta$ es el vector de parámetros desconocidos. Para resolver este problema de optimización no lineal, se implementa el algoritmo iterativo de **Descenso de Gradiente**. Este método busca el mínimo local de la función de costo actualizando los parámetros en la dirección opuesta al gradiente calculado. La regla de actualización en cada iteración $k$ está dada por:

$$\beta_{k+1} = \beta_k - \eta \cdot \nabla S(\beta_k)$$

Donde $\eta$ representa la tasa de aprendizaje, un hiperparámetro que controla la magnitud del ajuste en cada paso. El proceso iterativo continúa hasta que se alcanza un criterio de convergencia predefinido, momento en el cual se obtienen los estimadores de mínimos cuadrados condicionales que mejor ajustan el modelo a los datos observados.

In [40]:
def estimar_parametros_manual(serie, p, q, lr=1e-9, max_iter=1000):
    n_params = 1 + p + q
    params = np.zeros(n_params) + 0.001

    for _ in range(max_iter):
        grad = calcular_gradiente_manual(params, serie, p, q)
        params = params - (lr * grad)

    sse_final = calcular_sse_manual(params, serie, p, q)
    return params, sse_final

### 5. Criterio de Selección del Modelo (AIC)

Para la discriminación objetiva entre los distintos modelos candidatos y la determinación del orden óptimo $(p, q)$, se utiliza el Criterio de Información de Akaike (AIC). Este índice estadístico proporciona una medida de la calidad relativa del modelo, fundamentada en la teoría de la información, estableciendo un balance entre la bondad de ajuste y la complejidad del mismo (principio de parsimonia).

La formulación matemática del criterio se define mediante la siguiente expresión logarítmica:

$$AIC = T \ln(\hat{\sigma}^2) + 2k$$

Donde:
* $T$ representa el número de observaciones efectivas utilizadas en la estimación.
* $\hat{\sigma}^2$ es el estimador de máxima verosimilitud de la varianza del proceso de ruido blanco, calculado a partir de la suma de cuadrados de los residuos ($SSE$) como $\hat{\sigma}^2 = \frac{SSE}{T}$.
* $k$ denota el número total de parámetros estimados en el modelo ($p + q + 1$, incluyendo la constante).

El criterio penaliza la inclusión de parámetros adicionales mediante el término $2k$, compensando la reducción en la varianza del error y evitando así el sobreajuste del modelo a la serie de datos.

In [41]:
def calcular_aic(sse, n_obs, n_params):
    if sse <= 0: return float('inf')
    sigma2 = sse / n_obs
    aic = n_obs * math.log(sigma2) + 2 * n_params
    return aic

### 6. Ejecución y Selección Final

La etapa final del procedimiento de modelación estocástica integra los resultados de la estimación paramétrica para determinar el modelo óptimo que mejor describa el proceso generador de los datos. Esta fase se sustenta en la evaluación comparativa de los modelos candidatos sugeridos durante la etapa de identificación, específicamente las estructuras $AR(2)$, $AR(3)$, $AR(4)$ y $ARMA(2,1)$.

El criterio de decisión se fundamenta en el principio de parsimonia y la minimización de la pérdida de información de Kullback-Leibler. Para cada modelo candidato $M_i$ con estructura $(p, d, q)$, se calculan sus parámetros de máxima verosimilitud y el correspondiente valor del Criterio de Información de Akaike ($AIC_i$).

La regla de decisión estricta para la selección del modelo ganador se define formalmente como:

$$M_{óptimo} = \arg \min_{M_i \in \Omega} \{ AIC(M_i) \}$$

Donde $\Omega$ representa el conjunto de modelos candidatos evaluados. Aquel modelo que minimice el valor del AIC será seleccionado como el más adecuado, garantizando un equilibrio óptimo entre la bondad de ajuste (reducción de la varianza residual) y la complejidad del modelo (número de parámetros estimados).

In [48]:
def ejecutar_seleccion_final(datos_crudos):

    candidatos = [
        (2, 0, 0),
        (3, 0, 0),
        (4, 0, 0),
        (2, 0, 1)
    ]

    mejor_aic = float('inf')
    mejor_modelo = None
    mejor_params = None

    print(f"{'MODELO':<15} | {'SSE':<12} | {'AIC':<12}")
    print("-" * 45)

    for p, d, q in candidatos:
        try:
            datos_trabajo = diferenciar_manual(datos_crudos, d)

            params, sse = estimar_parametros_manual(datos_trabajo, p, q)

            k = len(params)
            T_eff = len(datos_trabajo) - max(p, q)
            aic_val = calcular_aic(sse, T_eff, k)

            print(f"ARIMA({p},{d},{q})    | {sse:.2f}   | {aic_val:.4f}")

            if aic_val < mejor_aic:
                mejor_aic = aic_val
                mejor_modelo = (p, d, q)
                mejor_params = params
        except:
            continue

    print("-" * 45)
    print(f"MODELO GANADOR: ARIMA{mejor_modelo}")
    print(f"AIC Mínimo: {mejor_aic:.4f}")

    print(f"PARÁMETROS ESTIMADOS:")
    nombres = ['Constante (c)'] + [f'AR({i+1})' for i in range(mejor_modelo[0])] + [f'MA({i+1})' for i in range(mejor_modelo[2])]

    for nombre, valor in zip(nombres, mejor_params):
        print(f"{nombre:<15}: {valor:.6f}")

if 'serie_solar' in locals():
    ejecutar_seleccion_final(serie_solar)

MODELO          | SSE          | AIC         
---------------------------------------------
ARIMA(2,0,0)    | 2081274.16   | 19276.9864
ARIMA(3,0,0)    | 2001911.90   | 19159.3167
ARIMA(4,0,0)    | 1957274.64   | 19089.6307
ARIMA(2,0,1)    | 2111822.40   | 19321.7668
---------------------------------------------
MODELO GANADOR: ARIMA(4, 0, 0)
AIC Mínimo: 19089.6307
PARÁMETROS ESTIMADOS:
Constante (c)  : 0.010931
AR(1)          : 0.575530
AR(2)          : 0.158144
AR(3)          : 0.102603
AR(4)          : 0.147206


### Conclusiones de la Etapa de Estimación

A partir de la ejecución del algoritmo de optimización y el análisis comparativo de los modelos candidatos, se determinó que el modelo **ARIMA(4,0,0)** minimiza de manera efectiva el Criterio de Información de Akaike (AIC), alcanzando un valor de **19,089.63**, el menor entre todas las especificaciones evaluadas.

Este resultado empírico valida la hipótesis planteada en la fase de **Identificación** respecto a la estructura de dependencia de la serie:

1.  **Validación del Límite Superior:** Durante la identificación, se sugirió el modelo $AR(4)$ como un "límite superior" debido a que el cuarto rezago se encontraba en la frontera de las bandas de significancia de la FAP. La estimación paramétrica confirma que la inclusión del término $\phi_4$ (estimado en $0.147$) aporta una reducción significativa en la varianza del error que compensa la penalización por complejidad del AIC.
2.  **Superioridad frente al AR(3):** Aunque el modelo $AR(3)$ captura gran parte de la estructura, la diferencia en el AIC respecto al modelo ganador ($19,159$ vs $19,089$) indica que truncar el modelo en el tercer rezago conlleva una pérdida de información relevante sobre el ciclo de las manchas solares.
3.  **Ineficacia del Componente de Media Móvil:** El modelo $ARMA(2,1)$ presentó el peor desempeño ($AIC \approx 19,321$). Esto corrobora que la persistencia de la serie se modela más adecuadamente mediante una estructura autorregresiva pura de mayor orden, en lugar de corregir mediante términos de media móvil.

En conclusión, se selecciona el modelo **ARIMA(4,0,0)** como la especificación óptima para proceder a las siguientes pruebas.